# <font style = "color:rgb(50,120,229)">Auto Focus Assignment</font>

Autofocus is implemented in all digital cameras these days. 

While using your phone camera, you may have noticed, the camera goes out of focus for a second or two, and the image looks blurry for a bit. The camera quickly performs some calculations and autofocuses to bring the subject in focus. 

In SLR cameras, autofocus is activated when we press the button half way through. You can see and hear parts of the lens moving as the camera tries to autofocus. 

Whether it is an SLR camera or your phone camera, autofocussing is done by taking a series of photos of the scene while changing the distance of the image sensor from the lens inside the camera.

In this assignment, we will find the sharpest image in a video squence of a static scene. In essence, we will do the computation necessary for autofocussing. 


In [2]:
# Import modules
import cv2
import matplotlib.pyplot as plt
from dataPath import DATA_PATH
import numpy as np
%matplotlib inline

In [3]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
matplotlib.rcParams['image.interpolation'] = 'bilinear'

# <font style = "color:rgb(50,120,229)">Measures of Sharpness</font>

How do we know if an image is sharp? What is a good measure of sharpness?

As you can imagine, an out of focus image is smooth and does not have large gradient. So some function of the gradient (first derivative) of an image should help you. 

A different measure could be based on the second order derivative of the image called the Laplacian. 

In this assignment, you to have to read one paper and a section of another paper to figure out the sharpest image in a video sequence. 

1. [Diatom autofocusing in brightheld microscopy: a comparative study](https://www.researchgate.net/profile/Jose-Luis-Pech-Pacheco/publication/3887632_Diatom_autofocusing_in_brightfield_microscopy_A_comparative_study/links/598c5303458515c333a828f5/Diatom-autofocusing-in-brightfield-microscopy-A-comparative-study.pdf)): This paper has several measures of sharpess. 

2. [Shape from Focus](https://graphics.stanford.edu/courses/cs348b-06/homework3/Nayar_CVPR92.pdf): This paper is about estimating the 3D shape of a scene using focus information. In Section 5, the author discusses a measure of focus. 

In the above papers, the focus is calculated over small windows. For our assignement, the focus measure needs to be calcualted over the entire image and not a small window. 





## <font style = "color:rgb(50,120,229)">Assignment Instructions</font>

In this assignment, you have to implement the following measures of focus:

1. **Variance of absolute values of Laplacian** - `var_abs_laplacian` function

2. **Sum Modified Laplacian (SML)** - `sum_modified_laplacian` function

You will also have to manually specify the ROI for the flower region in the frame so that instead of carrying out the auto focus measurement on the entire frame, it can be carried out only on the flower.

## <font style = "color:rgb(50,120,229)">Grading Rubric</font>

The assignment carries **30 marks** and the marks distribution is as follows.

1. Implementing *Variance of absolute values of Laplacian* method - **10 marks** (Autograded)
2. Implementing *Sum Modified Laplacian (SML)* method - **10 marks** (Autograded)
3. Overall submission - **10 marks** (Manually graded)

A total of **5 submissions** will be allowed.

## <font style = "color:rgb(50,120,229)">Important Functions</font>

Here are some important functions that you can use in this assignment.

1. [`cv2.filter2D`](https://docs.opencv.org/4.1.0/d4/d86/group__imgproc__filter.html#ga27c049795ce870216ddfb366086b5a04): Convolves an image with the kernel. The function applies an arbitrary linear filter to an image.

2. [`cv2.Laplacian`](https://docs.opencv.org/4.1.0/d4/d86/group__imgproc__filter.html#gad78703e4c8fe703d479c1860d76429e6): Calculates the Laplacian of an image.


#### <font style = "color:rgb(200,0,0)">Note</font>
Please do not look for code online!  

<hr />

In [4]:
# Implement Variance of absolute values of Laplacian - Method 1
# Input: image
# Output: Floating point number denoting the measure of sharpness of image

# Do NOT change the function name and definition

def var_abs_laplacian(image):
    # Calculate the Laplacian of the image
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    
    # Take the absolute value of the Laplacian
    abs_laplacian = np.abs(laplacian)
    
    # Calculate the variance of the absolute values
    variance = np.var(abs_laplacian)
    
    return variance


In [5]:
# Implement Sum Modified Laplacian - Method 2
# Input: image
# Output: Floating point number denoting the measure of sharpness of image

# Do NOT change the function name and definition

def sum_modified_laplacian(im):
    # Define the kernels for the modified Laplacian
    kernel_x = np.array([[0, 0, 0], [-1, 2, -1], [0, 0, 0]])
    kernel_y = np.array([[0, -1, 0], [0, 2, 0], [0, -1, 0]])

    # Apply the kernels to the image
    laplacian_x = cv2.filter2D(im, cv2.CV_64F, kernel_x)
    laplacian_y = cv2.filter2D(im, cv2.CV_64F, kernel_y)

    # Calculate the sum of the absolute values of the results
    sum_modified_laplacian = np.sum(np.abs(laplacian_x) + np.abs(laplacian_y))

    return sum_modified_laplacian


Let's have a look at the input video.

&nbsp;

<center>
<video controls width=800 src="https://www.dropbox.com/s/p3z0hbgo7sacqd1/focus-test.mp4?dl=1" type="video/mp4" />
</center>

In [6]:
# Read input video filename
filename = "./focus-test.mp4"

# Create a VideoCapture object
cap = cv2.VideoCapture(filename)

# Read first frame from the video
ret, frame = cap.read()

if not ret:
    print("Failed to read the video file.")
    cap.release()
    exit()

# Select ROI

roi = cv2.selectROI("Select ROI", frame, fromCenter=False, showCrosshair=True)
# Wait for a key press to close the window
cv2.waitKey(0)
cv2.destroyWindow("Select ROI")

# Extract ROI coordinates
x, y, w, h = roi
top, left, bottom, right = y, x, y + h, x + w

print(f"Top: {top}, Left: {left}, Bottom: {bottom}, Right: {right}")

Top: 46, Left: 418, Bottom: 649, Right: 1073


In [20]:
# Read input video filename
filename = './focus-test.mp4'

# Create a VideoCapture object
cap = cv2.VideoCapture(filename)

# Read first frame from the video
ret, frame = cap.read()

# Display total number of frames in the video
print("Total number of frames : {}".format(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

maxV1 = 0
maxV2 = 0

# Frame with maximum measure of focus
# Obtained using methods 1 and 2
bestFrame1 = 0 
bestFrame2 = 0 

# Frame ID of frame with maximum measure
# of focus
# Obtained using methods 1 and 2
bestFrameId1 = 0 
bestFrameId2 = 0 

# Specify the ROI for flower in the frame
# UPDATE THE VALUES BELOW
#top = 0
#left = 0
#bottom = frame.shape[0]
#right = frame.shape[1]

# Iterate over all the frames present in the video
while(ret):
    # Crop the flower region out of the frame
    flower = frame[top:bottom, left:right]
    # Get measures of focus from both methods
    val1 = var_abs_laplacian(flower)
    val2 = sum_modified_laplacian(flower) 
    frame_id = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    print("Frame: %d, VAR_LAP: %f, SML: %d" % (frame_id,val1,val2))
    
    # If the current measure of focus is greater 
    # than the current maximum
    if val1 > maxV1 :
        # Revise the current maximum
        maxV1 = val1
        # Get frame ID of the new best frame
        bestFrameId1 = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        # Revise the new best frame
        bestFrame1 = frame.copy()
        print("Frame ID of the best frame [Method 1]: {}".format(bestFrameId1))

    # If the current measure of focus is greater 
    # than the current maximum
    if val2 > maxV2 : 
        # Revise the current maximum
        maxV2 = val2
        # Get frame ID of the new best frame
        bestFrameId2 = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        # Revise the new best frame
        bestFrame2 = frame.copy()
        print("Frame ID of the best frame [Method 2]: {}".format(bestFrameId2))
        
    # Read a new frame
    ret, frame = cap.read()


print("================================================")
# Print the Frame ID of the best frame
print("Frame ID of the best frame [Method 1]: {}".format(bestFrameId1))
print("Frame ID of the best frame [Method 2]: {}".format(bestFrameId2))

# Release the VideoCapture object
cap.release()

# Stack the best frames obtained using both methods
out = np.hstack((bestFrame1, bestFrame2))

# Display the stacked frames
plt.figure()
plt.imshow(out[:,:,::-1])
plt.axis('off')

Total number of frames : 186
Frame: 1, VAR_LAP: 5.053682, SML: 6463981
Frame ID of the best frame [Method 1]: 1
Frame ID of the best frame [Method 2]: 1
Frame: 2, VAR_LAP: 4.587558, SML: 6336910
Frame: 3, VAR_LAP: 4.962127, SML: 6557052
Frame ID of the best frame [Method 2]: 3
Frame: 4, VAR_LAP: 4.074386, SML: 5922554
Frame: 5, VAR_LAP: 4.722467, SML: 6274964
Frame: 6, VAR_LAP: 4.447176, SML: 6254757
Frame: 7, VAR_LAP: 4.696886, SML: 6324512
Frame: 8, VAR_LAP: 4.518406, SML: 6307549
Frame: 9, VAR_LAP: 5.255025, SML: 6793638
Frame ID of the best frame [Method 1]: 9
Frame ID of the best frame [Method 2]: 9
Frame: 10, VAR_LAP: 4.433825, SML: 6351723
Frame: 11, VAR_LAP: 4.664687, SML: 6479974
Frame: 12, VAR_LAP: 4.430483, SML: 6324546
Frame: 13, VAR_LAP: 5.203155, SML: 6841338
Frame ID of the best frame [Method 2]: 13
Frame: 14, VAR_LAP: 4.325425, SML: 6262201
Frame: 15, VAR_LAP: 4.622311, SML: 6408725
Frame: 16, VAR_LAP: 4.175331, SML: 6185681
Frame: 17, VAR_LAP: 5.066851, SML: 6775063
Fr

(np.float64(-0.5), np.float64(2559.5), np.float64(719.5), np.float64(-0.5))

In [21]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [22]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


# <font style = "color:rgb(50,120,229)">Additional Exercise</font>

In this assignment you implemented 2 of the several measures of focus. Now, try to implement the rest of the measures of focus and see if your output matches the output you obtained using the above 2 methods. 

Which method do you think is the best one and why? Share your answers on the discussion forum.